### TMDB-Extraction - Utility Functions

In [1]:
# TMDB - API_KEY
api_key = '93c3c10cfb3d981195193c06b5e45d36'

In [2]:
import pandas as pd
import requests

In [3]:
#The Third API call to get the keywords

def movie_keywords(movie_id):
    url_keywords = ('https://api.themoviedb.org/3/movie/{}/keywords?api_key='+api_key).format(movie_id)
    response_url_keywords = requests.get(url_keywords).json()
    #time.sleep(1)
    keywords = [keyword['name'] for keyword in response_url_keywords['keywords']]

    return keywords

In [4]:
# Fourth API-call to get popular casts & crew

def cast_crew(movie_id):

    url = ('https://api.themoviedb.org/3/movie/{}/credits?api_key='+api_key+'&language=en-US').format(movie_id)
    response_crew_cast = requests.get(url).json()
  
    try:
        director_name = [i['name'] for i in response_crew_cast['crew'] if i['job']=='Director']
    except:
        director_name = []
    
    try:
        popularity_dict = {}
        df = pd.DataFrame()
        if response_crew_cast['cast']!=[] :
            for i in response_crew_cast['cast']:
                popularity_dict['name'] = i['name']
                popularity_dict['popularity'] = i['popularity']

                df = df.append(popularity_dict,ignore_index=True)

        if not df.empty:
            df = df.sort_values('popularity',ascending=False).head(5)
            popular_cast = list(df['name'].values)
        else:
            popular_cast = []
    except:
        popular_cast = []
        
        
    return director_name,popular_cast

In [5]:
# Final API - Call to club all the collected data into a Dataframe

def movie_details(max_pages=5):

    pageToken=1
    
    features = ['movie_id','movie_genre','title','overview','release_date','keywords','director_name','popular_cast']
    df = pd.DataFrame()
    
    while True:
        
        url = 'https://api.themoviedb.org/3/discover/movie?api_key='+api_key+'&language=en-US&'+f"&page={pageToken}"
        response_movie_id = requests.get(url).json()
            #time.sleep(1)

        for id in response_movie_id['results']:
            movie_id=id['id']
            
            if id['original_language']=='en':
                url_movie_details = ('https://api.themoviedb.org/3/movie/{}?api_key='+api_key+'&language=en-US').format(movie_id)
                response_movie_details = requests.get(url_movie_details).json()
                if response_movie_details['status'] =='Released':
                    movie_genre = [id['name'] for id in response_movie_details['genres'] if response_movie_details['genres']!=[] ]
                    title = response_movie_details['original_title']
                    overview = response_movie_details['overview']
                    release_date = response_movie_details['release_date']
                    keywords = movie_keywords(movie_id)
                    director_name,popular_cast = cast_crew(movie_id)

                df = df.append({'movie_id':movie_id,
                                'title':title,
                                'movie_genre':movie_genre,
                                'overview':overview,
                                'director_name' : director_name,
                                'Popular_cast':popular_cast,
                                'release_date':release_date,'keywords':keywords},ignore_index=True)

        pageToken+=1
        if pageToken>max_pages:
            break

    return df